In [2]:
#import pandas, numpy, os, matplotlib, seaborn, scrpy libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
path = r'C:\Users\Josh Abrams\Desktop\09-2022 Instacart Basket Analysis'

In [6]:
customers = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'customers.csv'))
ords_prods_merged = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_merged_user_flags.pkl'))
departments=pd.read_csv(os.path.join(path, 'Data', 'Prepared Data', 'departments_wrangled.csv'))


In [8]:
departments.head()

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [3]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [4]:
#4
#based on brief we can drop: firstname, surnam. not needed for any of the questions posed. Also anonymity?
#need user_id for merge, gender, state, age, dependents, fam_status, income for marketing questions in brief
#rename STATE to state, Gender to gender, Age to age, n_dependants to num_dependants, family_status to marital_status
customers = customers.drop(columns = ['First Name', 'Surnam'])
customers.rename(columns = {'Gender':'gender', 'STATE':'state', 'Age':'age', 'n_dependants':'num_dpdnts', 'fam_status':'marital_status'}, inplace = True)
customers.head()

,user_id,gender,state,age,date_joined,num_dpdnts,marital_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
#5a
#check for mixed data types
for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (customers[weird]) > 0:
        print (col)
#no mixed data types found

In [6]:
#5b
#find missing values by counting nulls
customers.isnull().sum()
#no nulls in data set


user_id           0
gender            0
state             0
age               0
date_joined       0
num_dpdnts        0
marital_status    0
income            0
dtype: int64

In [7]:
#5c
#check for d
#uplicate data
customersDuplicates = customers[customers.duplicated()]
customersDuplicates.shape
#no dupliacte values in data set

(0, 8)

In [8]:
#check data set after data wrangling
customers.head()

,user_id,gender,state,age,date_joined,num_dpdnts,marital_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
#6a
#preparing for merge
#checking user_id is same data type for merge
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   user_id         206209 non-null  int64 
 1   gender          206209 non-null  object
 2   state           206209 non-null  object
 3   age             206209 non-null  int64 
 4   date_joined     206209 non-null  object
 5   num_dpdnts      206209 non-null  int64 
 6   marital_status  206209 non-null  object
 7   income          206209 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 12.6+ MB


In [10]:
customers.rename(columns = {'user_id' : 'customer_id'}, inplace = True)

In [11]:
ords_prods_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 27 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   Unnamed: 0_x                      int64   
 1   order_id                          int64   
 2   customer_id                       int64   
 3   eval_set                          object  
 4   order_number                      int64   
 5   orders_day_of_week                int8    
 6   order_hour_of_day                 int32   
 7   days_since_prior_order            float64 
 8   product_id                        int64   
 9   add_to_cart_order                 int64   
 10  reordered                         int64   
 11  Unnamed: 0_y                      int64   
 12  product_name                      object  
 13  aisle_id                          int64   
 14  department_id                     int64   
 15  prices                            float64 
 16  _merge          

User_id will be used to join the data sets

In [12]:
#drop _merge column to use again
ords_prods_merged = ords_prods_merged.drop(columns = '_merge')
ords_prods_merged.shape

(32404859, 26)

In [13]:
#6b
#merge the data sets using left join. Keep everything in ord_prods, and map customer information to each purchase
cust_ord_combined = ords_prods_merged.merge(customers, on = 'customer_id', indicator= True)
cust_ord_combined['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [14]:
ords_prods_merged.shape

(32404859, 26)

In [15]:
customers.shape

(206209, 8)

In [16]:
cust_ord_combined.shape

(32404859, 34)

In [17]:
pd.options.display.max_columns = None
cust_ord_combined.head(100)
#looks right, size is right. 24 from ords_prods_merged, 8 from customers, minus 1 since joined on user_id, plus 1 for adding _merge

,Unnamed: 0_x,order_id,customer_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_days_since_previous_order,Frequency_flag,gender,state,age,date_joined,num_dpdnts,marital_status,income,_merge
0,0,2539329,1,prior,1,2,8,NaN,196,1,0,195,Soda,77,7,9.0,Mid-range product,Regularly busy,Regulary busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,1,2398795,1,prior,2,3,7,15.0,196,1,1,195,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2,473747,1,prior,3,3,12,21.0,196,1,1,195,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,3,2254736,1,prior,4,4,7,29.0,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,4,431534,1,prior,5,4,15,28.0,196,1,1,195,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,134,3317979,15,prior,5,4,15,17.0,14715,1,1,14715,Coconut Water,98,7,4.0,Low-range product,Least busy,Slowest days,Most orders,22,Regular customer,3.980556,Low spender,10.0,Frequent customer,Female,Indiana,69,6/3/2019,0,divorced/widowed,54313,both
96,136,2685110,15,prior,7,1,11,17.0,14715,3,1,14715,Coconut Water,98,7,4.0,Low-range product,Regularly busy,Busiest days,Most orders,22,Regular customer,3.980556,Low spender,10.0,Frequent customer,Female,Indiana,69,6/3/2019,0,divorced/widowed,54313,both
97,138,887727,15,prior,9,2,13,7.0,14715,1,1,14715,Coconut Water,98,7,4.0,Low-range product,Regularly busy,Regulary busy days,Most orders,22,Regular customer,3.980556,Low spender,10.0,Frequent customer,Female,Indiana,69,6/3/2019,0,divorced/widowed,54313,both
98,140,2600170,15,prior,11,2,9,14.0,14715,1,1,14715,Coconut Water,98,7,4.0,Low-range product,Regularly busy,Regulary busy days,Most orders,22,Regular customer,3.980556,Low spender,10.0,Frequent customer,Female,Indiana,69,6/3/2019,0,divorced/widowed,54313,both


In [18]:
#8 export
cust_ord_combined.to_pickle(os.path.join(path, 'Data','Prepared Data', 'full_instacart_data.pkl'))